In [ ]:
import json
import datetime
import requests
from dateutil import parser
from itertools import cycle
from pymongo import MongoClient
from bson.objectid import ObjectId

In [ ]:
client = MongoClient('mongodb://######################/')
db = client['SONAR_CHALLENGE']
collection = db["python_repositories_metadata"]

In [ ]:
tokens = ['tkn1',
          'tkn2',
          'tkn3',
          'tkn4',
          'tkn5']

In [ ]:
list_of_repositories = [line.split(None, 1)[1].rstrip() for line in open('./github_repos.txt', 'r')]

In [ ]:
def insert_projects_in_db(list_of_repo):
    
    for repository in list_of_repo:
        
        repo_init = {}
        
        repo_init['html_url'] = repo_url = repository[:-1] if repository.endswith('/') else repository
        repo_init['available'] = False
        repo_init['revised'] = False
        
        json_data_to_collection = json.dumps(repo_init)
        collection.insert_one(json.loads(json_data_to_collection))

In [ ]:
# insert_projects_in_db(list_of_repositories)

In [ ]:
def availability_process(list_of_repo):
    
    alternator = cycle(range(4))
    
    cursors = collection.find({'available':False,'revised':False})
        
    for repo in cursors:
        
        active_tkn = tokens[next(alternator)]
        
        headers = {'Authorization': 'token ' + active_tkn}
        repo_url = repo['html_url'].replace('http://github.com/','')
        repo_url = repo_url[:-1] if repo_url.endswith('/') else repo_url
        
        try:
            
            response = requests.get(f'https://api.github.com/repos/{repo_url}', headers=headers)
            response.raise_for_status()
            
            response_data = json.dumps(response.json())
            response_data_as_json = json.loads(response_data)        
                
            collection.update_one({'_id': ObjectId(repo['_id'])},
               {"$set": 
                {
                    'full_name': response_data_as_json['full_name'],
                    'description': response_data_as_json['description'],
                    'created_at': response_data_as_json['created_at'],
                    'pushed_at': response_data_as_json['pushed_at'],
                    'updated_at': response_data_as_json['updated_at'],
                    'size': response_data_as_json['size'],
                    'language': response_data_as_json['language'],
                    'forks': response_data_as_json['forks'],
                    'watchers': response_data_as_json['watchers'],
                    'stargazers_count': response_data_as_json['stargazers_count'],
                    'available': True,
                    'revised': True,
                }
               })

            
        except requests.exceptions.HTTPError as err:
            
            print("#"*50)
            print (" I am assessing: ", repo_url, " token: ", active_tkn)
            print (" Exception cause : ", err)
            print("#"*50)
            
            if 'Not Found' in str(err):
                
                collection.update_one({'_id': ObjectId(repo['_id'])},
                   {"$set": 
                    {
                        'available': False,
                        'revised': True,
                    }
                   })


In [ ]:
# availability_process(list_of_repositories)